# 管理历史记忆

对于附带历史记忆的对话，应该管理历史对话的长度，避免token爆炸的情况。

In [1]:
import shutup
shutup.please()

from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain

load_dotenv()
llm = ChatOpenAI(model='qwen2:7b')

直接给定用户输入，查看结果：

In [2]:
conversation = ConversationChain(llm=llm, verbose=True)

result = conversation.predict(input='hello')
result = conversation.predict(input='你叫什么名字？')

result = conversation.predict(input='感冒是一种什么病？')
result = conversation.predict(input='一般会有哪些症状？')
result = conversation.predict(input='吃什么药好得快？')
result



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: hello
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: hello
AI: Hello! How can I assist you today?
Human: 你叫什么名字？
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does

'在应对感冒时，治疗的主要目的是缓解症状和提高身体抵抗力，而不是快速消除病毒。以下是一些常用药物及其作用：\n\n1. **解热镇痛药**（如布洛芬或对乙酰氨基酚）：这些药物可以帮助降低体温、减轻疼痛和肌肉酸痛。请根据产品说明或医生建议使用，注意不要与含有相同成分的其他药物同时服用。\n\n2. **止咳糖浆**：用于缓解干咳症状，通常含有人工合成的镇咳药如右美沙芬（适用于成人）。\n\n3. **祛痰药**：如果感冒导致有痰，可使用乙酰半胱氨酸等祛痰药帮助清除呼吸道分泌物。这类药物通常通过稀释痰液来使它更容易咳出。\n\n4. **鼻腔喷雾或滴剂**：这些非处方药物可以用于缓解流鼻涕和鼻塞症状，如含盐水的冲洗、去充血剂（比如麻黄素）等。\n\n5. **维生素C补充剂**：虽然没有足够的科学证据证明维生素C能快速治愈感冒，但适量摄入维生素C可能有助于增强免疫系统。\n\n请记住，在使用任何药物之前，应该仔细阅读说明书并考虑个人健康状况。如果你有特定的健康问题、正在服用其他药物或有任何疑虑，请咨询医生或药师的意见。在自我治疗时，应避免自行调整剂量或同时使用多种药物，以免造成不必要的副作用或相互作用。\n\n总的来说，快速“好得快”的方法主要是通过适当的药物管理来缓解症状和充足的休息与水分摄入。保持健康的生活习惯和增强免疫力也是预防感冒复发的关键。'

这里，Langchain可以指定保留的hitstory轮数，比如新的对话仅向前查看两轮对话的历史记录，从而减少token消耗。

In [3]:
from langchain.memory import (ConversationBufferMemory,
                              ConversationBufferWindowMemory,
                              ConversationSummaryMemory,
                              ConversationSummaryBufferMemory)

memory = ConversationBufferWindowMemory(k=2)  # 仅保留两轮对话
conversation = ConversationChain(llm=llm, memory=memory, verbose=True)

In [4]:
result = conversation.predict(input='我叫路人甲')
result = conversation.predict(input='你叫什么名字？')
result = conversation.predict(input='感冒是一种什么病？')
result = conversation.predict(input='我叫什么名字？')
result



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 我叫路人甲
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 我叫路人甲
AI: 你好，"路人甲"！很高兴与你交谈。您有什么问题或想聊些什么？我在这里帮助解答您的疑问或是提供相关信息。
Human: 你叫什么名字？
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its contex

'你可以告诉我你想要我怎么称呼你，比如可以用一个昵称或者你的真实姓名。如果你没有特定的偏好，那我们可以继续以“您”来互相问候和交流。请告诉我你的决定，这样我们就能更好地进行对话了。'

可以看到，历史记忆仅保留了两轮。

其他常见的历史管理方式：

```python
memory = ConversationBufferMemory()  # 保留完整会话（默认）
memory = ConversationBufferWindowMemory(k=2)  # 保留前k轮会话
memory = ConversationSummaryMemory(llm=llm)  # 总结前面的对话内容
memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)  # 超过最大token数量的会话，会被总结
```

除了完整的会话，其他形式都会有一定的信息损失，需要进行取舍。

In [5]:
memory = ConversationSummaryMemory(llm=llm) 
conversation = ConversationChain(llm=llm, memory=memory, verbose=True)
result = conversation.predict(input='我叫路人甲')
result = conversation.predict(input='你叫什么名字？')
result = conversation.predict(input='感冒是一种什么病？')
result = conversation.predict(input='我叫什么名字？')
result



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 我叫路人甲
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
The human introduces themselves as "路人甲" (literally meaning "random person A") and the AI greets them warmly, addressing them as "路人大甲", expressing happiness to converse with them. The AI identifies itself as a smart assistant ready to provide assistance.
Human: 你叫什么名字？
AI:

> Finished chain.


> Entering new Conver

'您的名字是“路人甲”，您在对话中这样介绍自己。而我被称作“AI小助手”。如果您还有其他问题或者需要帮助，随时可以问我哦！'

可以看到，这里内容被AI自动总结了，但是总结的过程是英文，所以会存在信息失真的情况。